## Requirements
- openai （需要 1.x 版）
- Azure OpenAI 助理目前位於瑞典中部、美國東部 2 和澳大利亞東部。 如需這些區域中模型可用性的詳細資訊，請參閱 [模型指南](https://learn.microsoft.com/zh-tw/azure/ai-services/openai/concepts/models#assistants-preview)
- API Version 在 2024-02-15-preview 之後
- 模型：`gpt-35-turbo (1106)` 或 `gpt-4 (1106-preview)` 

In [86]:
from pydantic_settings import BaseSettings
from typing import Optional

class BingSearchSettings(BaseSettings):
    BING_SEARCH_API_KEY: str
    BING_SEACH_API_ENDPOINT: str = "https://api.bing.microsoft.com/v7.0/search"


class AOAISettings(BaseSettings):
    AOAI_API_KEY: str
    AOAI_API_VERSION: str
    AOAI_API_ENDPOINT: str
    AOAI_GPT3_MODEL: str
    AOAI_GPT4_MODEL: Optional[str] = ""
    AOAI_EMBEDDING_MODEL: str


class Env(AOAISettings, BingSearchSettings):
    class Config:
        env_file = ".env"

env = Env()

In [87]:
from openai import AzureOpenAI

aoai = AzureOpenAI(
    api_key=env.AOAI_API_KEY,
    api_version='2024-02-15-preview',
    azure_endpoint=env.AOAI_API_ENDPOINT,
)

## Helper Functions

In [88]:
import json
import time

def poll_run_till_completion(
    client: AzureOpenAI,
    thread_id: str,
    run_id: str,
    available_functions: dict,
    verbose: bool,
    max_steps: int = 10,
    wait: int = 3,
) -> None:
    """
    Poll a run until it is completed or failed or exceeds a certain number of iterations (MAX_STEPS)
    with a preset wait in between polls

    @param client: OpenAI client
    @param thread_id: Thread ID
    @param run_id: Run ID
    @param assistant_id: Assistant ID
    @param verbose: Print verbose output
    @param max_steps: Maximum number of steps to poll
    @param wait: Wait time in seconds between polls

    """

    if (client is None and thread_id is None) or run_id is None:
        print("Client, Thread ID and Run ID are required.")
        return
    try:
        cnt = 0
        while cnt < max_steps:
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            if verbose:
                print("Poll {}: {}".format(cnt, run.status))
            cnt += 1
            
            # 需要呼叫外部工具
            if run.status == "requires_action":
                tool_responses = []
                if (
                    run.required_action is not None and
                    run.required_action.type == "submit_tool_outputs"
                    and run.required_action.submit_tool_outputs.tool_calls is not None
                ):
                    tool_calls = run.required_action.submit_tool_outputs.tool_calls

                    for call in tool_calls:
                        if call.type == "function":
                            if call.function.name not in available_functions:
                                raise Exception("Function requested by the model does not exist")
                            function_to_call = available_functions[call.function.name]
                            tool_response = function_to_call(**json.loads(call.function.arguments))
                            tool_responses.append({"tool_call_id": call.id, "output": tool_response if isinstance(tool_response, str) else json.dumps(tool_response)})

                run = client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread_id, run_id=run.id, tool_outputs=tool_responses
                )
            if run.status == "failed":
                print("Run failed.")
                break
            if run.status == "completed":
                break
            time.sleep(wait)

    except Exception as e:
        print(e)


In [117]:
from pathlib import Path
from typing import Any

def retrieve_and_print_messages(
    client: AzureOpenAI, thread_id: str, verbose: bool, out_dir: Optional[str] = None
) -> Any:
    """
    Retrieve a list of messages in a thread and print it out with the query and response

    @param client: OpenAI client
    @param thread_id: Thread ID
    @param verbose: Print verbose output
    @param out_dir: Output directory to save images
    @return: Messages object

    """

    if client is None and thread_id is None:
        print("Client and Thread ID are required.")
        return None
    try:
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        display_role = {"user": "User: ", "assistant": "Assistant: "}

        # prev_role = None

        # if verbose:
            # print("\n\nCONVERSATION:")

        qa = []

        for message in messages.data:
            if message.role == "assistant":
                qa.append(message)
            if message.role == "user":
                qa.append(message)
                break

        for md in reversed(qa):
            # if prev_role == "assistant" and md.role == "user" and verbose:
                # print("------ \n")

            for mc in md.content:
                # Check if valid text field is present in the mc object
                if mc.type == "text":
                    txt_val = mc.text.value
                # Check if valid image field is present in the mc object
                elif mc.type == "image_file":
                    image_data = client.files.content(mc.image_file.file_id)
                    if out_dir is not None:
                        out_dir_path = Path(out_dir)
                        if out_dir_path.exists():
                            image_path = out_dir_path / (mc.image_file.file_id + ".png")
                            with image_path.open("wb") as f:
                                f.write(image_data.read())

                if verbose:
                    # if prev_role == md.role:
                        # print(txt_val)
                    # else:
                    print("{}:\n{}".format(display_role[md.role], txt_val))
            # prev_role = md.role
        print("=" * 100)
        return messages
    except Exception as e:
        print(e)
        return None


In [118]:
from openai.types.beta.assistant import Assistant
from typing import Iterable
from openai.types.beta import AssistantToolParam


def create_assistant(aoai: AzureOpenAI, name:str, instructions:str, model: str = env.AOAI_GPT4_MODEL if env.AOAI_GPT4_MODEL else env.AOAI_GPT3_MODEL, tools: Iterable[AssistantToolParam] = []) -> Assistant:
    return aoai.beta.assistants.create(name=name, instructions=instructions, model=model, tools=tools)

In [119]:
from openai.types.beta.assistant import Assistant
from time import sleep

def chat_with_assistant(assistant: Assistant, aoai: AzureOpenAI, available_functions: dict = {}, verbose: bool = False):
    """
    Chat with the assistant

    @param assistant: Assistant object
    @param aoai: OpenAI client
    @param available_functions: Dictionary of available functions
    @param verbose: Print verbose output

    """

    if assistant is None:
        print("Assistant is required.")
        return
    
    if aoai is None:
        print("OpenAI client is required.")
        return
        
    # Create a thread
    thread = aoai.beta.threads.create()

    while True:
        user_input = input("User: ")
        if user_input == "exit":
            break

        aoai.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_input,
        )


        # Run the thread
        run = aoai.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant.id,
        )

        # Retrieve the status of the run
        start_time = time.perf_counter()
        poll_run_till_completion(
            client=aoai,
            thread_id=thread.id,
            run_id=run.id,
            available_functions=available_functions,
            verbose=verbose,
        )
        end_time = time.perf_counter()
        if verbose:
            print(f"Time taken: {end_time - start_time:.2f} seconds")

        # Retrieve and print messages
        retrieve_and_print_messages(client=aoai, thread_id=thread.id, verbose=verbose)

        sleep(2)
      

## Code Interceptor
- [Reference](https://learn.microsoft.com/zh-tw/azure/ai-services/openai/how-to/code-interpreter?tabs=python)
- 需要額外的 [Pricing](https://azure.microsoft.com/zh-tw/pricing/details/cognitive-services/openai-service/)

In [120]:
from typing import Iterable
from openai.types.beta import AssistantToolParam


def create_math_assistant(
    model: str = env.AOAI_GPT4_MODEL if env.AOAI_GPT4_MODEL else env.AOAI_GPT3_MODEL,
):
    name = "Math Assist"
    instructions = (
        "You are an AI assistant that can write code to help answer math questions."
    )
    tools: Iterable[AssistantToolParam] = [{"type": "code_interpreter"}]

    return create_assistant(aoai, name, instructions, tools=tools, model=model)


def chat_with_math_assistant(
    aoai: AzureOpenAI,
    model: str = env.AOAI_GPT4_MODEL if env.AOAI_GPT4_MODEL else env.AOAI_GPT3_MODEL,
    verbose: bool = False,
):
    assistant = create_math_assistant(model=model)
    chat_with_assistant(aoai=aoai, assistant=assistant, verbose=verbose)

## Assistant Function Calling with Bing Search

- [Reference](https://github.com/Azure-Samples/azureai-samples/blob/main/scenarios/Assistants/function_calling/assistants_function_calling_with_bing_search.ipynb)

In [132]:
from typing import Iterable
import requests
from openai.types.beta import AssistantToolParam

def bing_search(query:str)->list:
    headers = {"Ocp-Apim-Subscription-Key": env.BING_SEARCH_API_KEY}
    params = {"q": query, "textDecorations": False}
    response = requests.get(env.BING_SEACH_API_ENDPOINT, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    output = []

    for result in search_results["webPages"]["value"]:
        output.append({"title": result["name"], "link": result["url"], "snippet": result["snippet"]})

    return output

def create_bing_search_assistant(
    model: str = env.AOAI_GPT4_MODEL if env.AOAI_GPT4_MODEL else env.AOAI_GPT3_MODEL,
):
    name = "Bing Search Assist"

    instructions = """You are an assistant designed to help people answer questions.

    You have access to query the web using Bing Search. You should call bing search whenever a question requires up to date information or could benefit from web data.
    """


    tools:Iterable[AssistantToolParam] = [
        {
            "type": "function",
            "function": {
                "name": "search_bing",
                "description": "Searches bing to get up-to-date information from the web.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The search query",
                        }
                    },
                    "required": ["query"],
                },
            },
        }
    ]

    return create_assistant(aoai, name, instructions, tools=tools, model=model)

def chat_with_bing_search_assistant(
    aoai: AzureOpenAI,
    model: str = env.AOAI_GPT4_MODEL if env.AOAI_GPT4_MODEL else env.AOAI_GPT3_MODEL,
    verbose: bool = False,
):
    assistant = create_bing_search_assistant(model=model)
    chat_with_assistant(aoai=aoai, assistant=assistant, available_functions={"search_bing": bing_search}, verbose=verbose)

## Multi Agent
- [Reference](https://github.com/Azure-Samples/azureai-samples/tree/main/scenarios/Assistants/multi-agent)

In [136]:
from openai.types.beta import Assistant, AssistantToolParam

def create_proxy_agent(
    assistants: list[Assistant],
    model: str = env.AOAI_GPT4_MODEL if env.AOAI_GPT4_MODEL else env.AOAI_GPT3_MODEL,
):
    """
    This agent facilitates the conversation between the user and other agents, ensuring successful completion of the task.
    """

    name = "Proxy Agent"
    instructions = f"You are an AI assistant that can help users answer questions using {', '.join([assistant.name.__str__() for assistant in assistants])}."

    tools: Iterable[AssistantToolParam] = []
    has_code_interpreter = False
    for assistant in assistants:
        for tool in assistant.tools:
            if tool.type == "function":
                tools.append(
                    {
                        "type": "function",
                        "function": tool.model_dump(mode="json")["function"],
                    }
                )

            if tool.type == "code_interpreter" and not has_code_interpreter:
                has_code_interpreter = True
                tools.append({"type": "code_interpreter"})

    return create_assistant(aoai, name, instructions, tools=tools, model=model)


def chat_with_proxy_assistant(
    aoai: AzureOpenAI,
    assistants: list[Assistant],
    model: str = env.AOAI_GPT4_MODEL if env.AOAI_GPT4_MODEL else env.AOAI_GPT3_MODEL,
    available_functions: dict = {},
    verbose: bool = False,
):
    assistant = create_proxy_agent(assistants=assistants, model=model)
    chat_with_assistant(assistant, aoai, verbose=verbose, available_functions=available_functions)

In [126]:
chat_with_math_assistant(aoai, verbose=True, model=env.AOAI_GPT3_MODEL)

Poll 0: queued
Poll 1: in_progress
Poll 2: completed
Time taken: 7.14 seconds
User: :
Ali’s class has a capacity of 120 students.  Each of John’s classes has a capacity of 120/8 = 15 students.  The total capacity of John’s two classes is?
Assistant: :
The total capacity of John's two classes can be calculated by multiplying the capacity of each class by the number of classes. Since each of John's classes has a capacity of 15 students, and he has two classes, the total capacity is:

Total capacity = (Capacity of 1 class) x (Number of classes)

Let's calculate it.
Assistant: :
The total capacity of John's two classes is 30 students.
Poll 0: queued
Poll 1: completed
Time taken: 3.74 seconds
User: :
最近台北市有什麼新聞？
Assistant: :
很抱歉，由於這個環境被禁用了網絡訪問功能，所以無法提供即時的新聞。建議您使用網絡訪問功能來瀏覽台北市的新聞網站或者使用新聞應用程序來獲取最新的新聞資訊。


In [127]:
chat_with_bing_search_assistant(aoai, verbose=True, model=env.AOAI_GPT3_MODEL)

Poll 0: queued
Poll 1: completed
Time taken: 3.74 seconds
User: :
Ali’s class has a capacity of 120 students.  Each of John’s classes has a capacity of 120/8 = 15 students.  The total capacity of John’s two classes is?
Assistant: :
John's class has a capacity of 15 students. Since John has two classes, the total capacity of John's two classes is 15 * 2 = 30 students.
Poll 0: in_progress
Poll 1: requires_action
Poll 2: in_progress
Poll 3: in_progress
Poll 4: in_progress
Poll 5: completed
Time taken: 19.05 seconds
User: :
最近台北市有什麼新聞？
Assistant: :
根據最新的資料，台北市的新聞包括：

1. 《自由時報》報導稱「台灣自古屬中國」王定宇稱「古态阿儸」：唬爛就是唬爛。過去吸蟹區縣市府還大使北市10大奙音車投訴熱點未設定固定式燈照相... (來源: [自由時報 - 即時 - 自由電子報](https://news.ltn.com.tw/list/breakingnews/Taipei))

2. 《聯合報》報導稱台北市今天下午大雷雨，其中最大雨量為大安區台灣大學站，時雨量約50.5毫米。台北市災防截圖顯示，昨天的降雨量約30分，也接報4件積水通報，其中建高一度洪水...(來源: [聯合新聞網](https://udn.com/news/story/7323/8038970))

3. 《自由時報》報導稱台北市政府5月初通過的「台北市興岩社福大業」由台北市社會局主導、新工處代辦工程，昨天的短期豪降雨，C杆頂樓排水不及，結果陽台間小雨、電梯笥水... (來源: [自由時報 - 大台北 - 自由電子報](https://news.ltn.co

In [139]:
math_assistant = create_math_assistant(env.AOAI_GPT3_MODEL)
bing_search_assistant = create_bing_search_assistant(env.AOAI_GPT3_MODEL)
available_functions = {"search_bing": bing_search}

chat_with_proxy_assistant(aoai, verbose=True, model=env.AOAI_GPT3_MODEL, assistants=[math_assistant, bing_search_assistant], available_functions=available_functions)

Poll 0: queued
Poll 1: in_progress
Poll 2: in_progress
Poll 3: in_progress
Poll 4: completed
Time taken: 13.83 seconds
User: :
Ali’s class has a capacity of 120 students.  Each of John’s classes has a capacity of 120/8 = 15 students.  The total capacity of John’s two classes is?
Assistant: :
The total capacity of John's two classes is 30 students.
Poll 0: queued
Poll 1: requires_action
Poll 2: in_progress
Poll 3: completed
Time taken: 11.94 seconds
User: :
最近台北市有什麼新聞？
Assistant: :
抱歉，我無法直接提供最新的台北市新聞。您可以使用網上新聞網站或新聞應用程式來瞭解台北市的最新新聞。常見的新聞網站和應用程式包括自由時報、中央社、蘋果日報和Yahoo新聞等。您可以在這些平台上搜尋「台北市最新新聞」以獲取最新的新聞資訊。
